Code to get 3x3 transformation matrix between tracking data and groundturth helmet bounding box

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import matplotlib.pyplot as plt
import cv2
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sideline_image = glob.glob('../../kaggle/input/nfl-impact-detection/images/*Sideline*')

In [ ]:
train_tracking = pd.read_csv('/kaggle/input/nfl-impact-detection/train_player_tracking.csv')

In [ ]:
train_label = pd.read_csv('/kaggle/input/nfl-impact-detection/train_labels.csv')

In [ ]:
train_tracking_frame = train_tracking[(train_tracking.gameKey == 57583) & (train_tracking.event == 'ball_snap')]

In [ ]:
train_tracking_frame ['x'] -= train_tracking_frame['x'].median()

In [ ]:
train_label_frame = train_label[(train_label['view'] == 'Sideline') & (train_label['gameKey'] == 57583) & (train_label['frame'] == 1)]

In [ ]:
merged_df = pd.merge(train_tracking_frame,train_label_frame,left_on='player',right_on='label')

In [ ]:
trakcing_coordinate = np.float32(list(zip(merged_df['x'],53.33-merged_df['y']))).reshape(-1,1,2)

In [ ]:
label_coordinate =  np.float32(list(zip(merged_df['left']+merged_df['width']/2,merged_df['top']-merged_df['height']/2))).reshape(-1,1,2)

In [ ]:
M, mask = cv2.findHomography( label_coordinate,trakcing_coordinate)

In [ ]:
transformed_coordinate =  cv2.perspectiveTransform(label_coordinate, M)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(transformed_coordinate[:,:,0],transformed_coordinate[:,:,1])
plt.scatter(merged_df['x'],53.33-merged_df['y'])

plt.legend(['Coordinate wrapped from helmet box','Ground truth tracking data'])